In [373]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import scipy.optimize as opt
from scipy.stats import skew
from scipy.stats import boxcox
from scipy.stats.stats import pearsonr
from sklearn.preprocessing import LabelEncoder
from scipy.stats import f_oneway

In [374]:
data=pd.read_csv('train.csv')
data.iloc[:,1].fillna(data.iloc[:,1].mean(), inplace=True)
data.iloc[:,3].fillna(data.iloc[:,3].mean(), inplace=True)
data.iloc[:,5].fillna(data.iloc[:,5].mean(), inplace=True)
data.loc[data.iloc[:,2]=='LF',['Item_Fat_Content']]='low fat'
data.loc[data.iloc[:,2]=='reg',['Item_Fat_Content']]='regular'
data.iloc[:,2]=data.iloc[:,2].str.lower()
data['Item_Type_modified']=data['Item_Identifier'].apply(lambda x: x[0:2])
data['Item_Type_modified']=data['Item_Type_modified'].map({'FD':'food', 'DR':'drink', 'NC':'non-consumable'})
data.drop(axis=1,labels='Item_Type', inplace=True)
data.iloc[:,6]=2013-data.iloc[:,6]
data.loc[(data['Item_Visibility'] == 0),'Item_Visibility'] = data.loc[(data['Item_Visibility'] == 0),'Item_Visibility'].apply(lambda x: data['Item_Visibility'].mean())
data.iloc[:,7].fillna('Medium', inplace=True)
data.loc[data.iloc[:,-1]=='non-consumable',['Item_Fat_Content']]='inedible'
data=pd.get_dummies(data, columns=['Item_Fat_Content','Item_Type_modified','Outlet_Identifier','Outlet_Establishment_Year','Outlet_Size','Outlet_Location_Type','Outlet_Type'])
data

,Item_Identifier,Item_Weight,Item_Visibility,Item_MRP,Item_Outlet_Sales,Item_Fat_Content_inedible,Item_Fat_Content_low fat,Item_Fat_Content_regular,Item_Type_modified_drink,Item_Type_modified_food,...,Outlet_Size_High,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 1,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,FDA15,9.300000,0.016047,249.8092,3735.1380,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,DRC01,5.920000,0.019278,48.2692,443.4228,0.0,0.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,FDN15,17.500000,0.016760,141.6180,2097.2700,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
3,FDX07,19.200000,0.066132,182.0950,732.3800,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
4,NCD19,8.930000,0.066132,53.8614,994.7052,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
5,FDP36,10.395000,0.066132,51.4008,556.6088,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
6,FDO10,13.650000,0.012741,57.6588,343.5528,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
7,FDP10,12.857645,0.127470,107.7622,4022.7636,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
8,FDH17,16.200000,0.016687,96.9726,1076.5986,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
9,FDU28,19.200000,0.094450,187.8214,4710.5350,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [375]:
data=data[[u'Item_Identifier', u'Item_Weight', u'Item_Visibility', u'Item_MRP',
       u'Item_Fat_Content_inedible',
       u'Item_Fat_Content_low fat', u'Item_Fat_Content_regular',
       u'Item_Type_modified_drink', u'Item_Type_modified_food',
       u'Item_Type_modified_non-consumable', u'Outlet_Identifier_OUT010',
       u'Outlet_Identifier_OUT013', u'Outlet_Identifier_OUT017',
       u'Outlet_Identifier_OUT018', u'Outlet_Identifier_OUT019',
       u'Outlet_Identifier_OUT027', u'Outlet_Identifier_OUT035',
       u'Outlet_Identifier_OUT045', u'Outlet_Identifier_OUT046',
       u'Outlet_Identifier_OUT049', u'Outlet_Establishment_Year_4',
       u'Outlet_Establishment_Year_6', u'Outlet_Establishment_Year_9',
       u'Outlet_Establishment_Year_11', u'Outlet_Establishment_Year_14',
       u'Outlet_Establishment_Year_15', u'Outlet_Establishment_Year_16',
       u'Outlet_Establishment_Year_26', u'Outlet_Establishment_Year_28',
       u'Outlet_Size_High', u'Outlet_Size_Medium', u'Outlet_Size_Small',
       u'Outlet_Location_Type_Tier 1', u'Outlet_Location_Type_Tier 2',
       u'Outlet_Location_Type_Tier 3', u'Outlet_Type_Grocery Store',
       u'Outlet_Type_Supermarket Type1', u'Outlet_Type_Supermarket Type2',
       u'Outlet_Type_Supermarket Type3',u'Item_Outlet_Sales']]
data

,Item_Identifier,Item_Weight,Item_Visibility,Item_MRP,Item_Fat_Content_inedible,Item_Fat_Content_low fat,Item_Fat_Content_regular,Item_Type_modified_drink,Item_Type_modified_food,Item_Type_modified_non-consumable,...,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 1,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Item_Outlet_Sales
0,FDA15,9.300000,0.016047,249.8092,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,3735.1380
1,DRC01,5.920000,0.019278,48.2692,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,443.4228
2,FDN15,17.500000,0.016760,141.6180,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2097.2700
3,FDX07,19.200000,0.066132,182.0950,0.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,732.3800
4,NCD19,8.930000,0.066132,53.8614,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,994.7052
5,FDP36,10.395000,0.066132,51.4008,0.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,556.6088
6,FDO10,13.650000,0.012741,57.6588,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,343.5528
7,FDP10,12.857645,0.127470,107.7622,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,4022.7636
8,FDH17,16.200000,0.016687,96.9726,0.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1076.5986
9,FDU28,19.200000,0.094450,187.8214,0.0,0.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,4710.5350


In [376]:
X=data.iloc[:,1:-1]
y=data.iloc[:,-1]
m=X.shape[0]
X=X.values
X = np.insert(X, 0, values=np.ones(m), axis=1)
n=X.shape[1]
y=y.values.reshape((8523,1))
theta=np.zeros((1,n))
X.shape, y.shape, theta.shape, m

((8523, 39), (8523, 1), (1, 39), 8523)

In [402]:
def J(theta,X,y, alpha=1):
    X=np.matrix(X)
    y=np.matrix(y)
    theta=np.matrix(theta)
    return np.sum(np.power(X*theta.T-y,2))/(2*m) + alpha*np.sum(np.power(theta,2))
J(theta,X,y)

3834910.2913576914

In [403]:
def grad(theta, X,y, alpha=1):
    X=np.matrix(X)
    y=np.matrix(y)
    theta=np.matrix(theta)
    grad=np.matrix(np.zeros(n))
    for j in range(n):
        grad[:,j]=np.sum(np.multiply((X*theta.T-y),X[:,j])/(m)) + alpha*theta[:,j]/m
    return grad
        

In [404]:
from scipy.optimize import fmin_tnc
theta_new=fmin_tnc(J,theta,grad,(X,y,0.05))
theta_new=np.matrix(theta_new[0])
theta_new

matrix([[ -61.5214594 ,   -4.59961563,  -46.96222518,   15.33723343,
          -32.87509123,  -38.1639365 ,    9.52579994,  -18.38448489,
          -10.25753492,  -32.87509123, -453.57753798,   -9.38316236,
           -2.75692969,  -78.47891764, -432.79347295,  653.44941598,
          124.34573806,  -85.24764701,  148.48086235,   88.00813447,
          -78.47891764,   -2.75692969,  124.34573806,  -85.24764701,
           88.00813447, -453.57753798,  148.48086235,   -9.38316236,
          196.38835678,   -9.38316236,   99.93078322, -153.57968686,
         -190.58384697,   36.76938705,   89.10375058, -976.22202009,
          267.23456578,  -78.47891764,  653.44941598]])

In [405]:
J(theta_new,X,y, 0.05)

772943.10114692152

In [406]:
test=pd.read_csv('test.csv')
test.iloc[:,1].fillna(test.iloc[:,1].mean(), inplace=True)
test.iloc[:,3].fillna(test.iloc[:,3].mean(), inplace=True)
test.iloc[:,5].fillna(test.iloc[:,5].mean(), inplace=True)
test.loc[test.iloc[:,2]=='LF',['Item_Fat_Content']]='low fat'
test.loc[test.iloc[:,2]=='reg',['Item_Fat_Content']]='regular'
test.iloc[:,2]=test.iloc[:,2].str.lower()
test['Item_Type_modified']=test['Item_Identifier'].apply(lambda x: x[0:2])
test['Item_Type_modified']=test['Item_Type_modified'].map({'FD':'food', 'DR':'drink', 'NC':'non-consumable'})
test.drop(axis=1,labels='Item_Type', inplace=True)
test.iloc[:,6]=2013-test.iloc[:,6]
test.loc[(test['Item_Visibility'] == 0),'Item_Visibility'] = test.loc[(test['Item_Visibility'] == 0),'Item_Visibility'].apply(lambda x: test['Item_Visibility'].mean())
test.iloc[:,7].fillna('Medium', inplace=True)
test.loc[test.iloc[:,-1]=='non-consumable',['Item_Fat_Content']]='inedible'
test=pd.get_dummies(test, columns=['Item_Fat_Content','Item_Type_modified','Outlet_Identifier','Outlet_Establishment_Year','Outlet_Size','Outlet_Location_Type','Outlet_Type'])
test=test.iloc[:,1:].values
test=np.insert(test, 0, np.ones(test.shape[0]), axis=1)
test=np.matrix(test)

In [407]:
sales=test*theta_new.T
sales=sales.reshape((5681))
sales.shape

(1, 5681)

In [408]:
r1=np.vstack((item,outlet))
r2=np.vstack((r1,sales))
result=pd.DataFrame(r2.T, columns=['Item_Identifier','Outlet_Identifier','Item_Outlet_Sales'])
result.to_csv('sampleSubmission.csv',index=False)